In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

In [ ]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error

import math
import pandas as pd
import operator
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from scipy import stats
import random
import copy

from scipy.stats import norm, kurtosis
from sklearn.metrics import make_scorer

from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers, models, optimizers, initializers, regularizers, constraints, losses
from keras.models import Sequential, Model, load_model, Input
from keras.layers import (Dense, Concatenate, BatchNormalization, Activation, Add,
                          concatenate, Dropout, AlphaDropout, Reshape, Layer, Multiply, Lambda)
from keras.layers import (Dense, Concatenate, BatchNormalization, Activation, Add,
                          concatenate, Dropout, AlphaDropout, Reshape, Layer, Multiply)
from keras.layers import (Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, 
                          GlobalMaxPooling1D, SeparableConv1D, MaxPool1D, AveragePooling1D, 
                          SeparableConv1D, AtrousConvolution1D)
from keras.layers import (Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, 
                          GlobalMaxPooling2D, SeparableConv2D, MaxPool2D, AveragePooling2D, 
                          SeparableConv2D, AtrousConvolution2D)
from keras.layers import LSTM, GRU, Bidirectional
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.losses import binary_crossentropy,  kullback_leibler_divergence, mean_squared_error

def mish(x):
    return x*K.tanh(K.softplus(x))

def decay(epoch, steps=100):
    initial_lrate = 1e-3
    drop = 0.9
    epochs_drop = 25
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    lrate = max(lrate, 5e-5)
    return lrate

es = EarlyStopping(patience=25, restore_best_weights=True, monitor='val_total_loss')
lrs = LearningRateScheduler(decay, verbose=2)

In [ ]:
def grap_year(x):
    y, _, _ = x.split('-')
    return int(y)

def grap_date(x):
    y = x[5:10].replace('-','')
    return y

def grap_hour(x):
    return x[11:13]

def custom_loss(y_true, y_pred):
    res = 0
    cnt = 1
    for i in range(0, len(y_true), 24):
        yt = y_true[i:i+24]
        yp = y_pred[i:i+24]
        a = np.abs(yt-yp)
        c = 113
        S = np.sum(yt)
        res += np.sum(a*yt/(S*c))
        cnt += 1
    return res/cnt

def custom_loss_nn(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    res = 0
    cnt = 1
    for i in range(0, len(y_true), 24):
        yt = y_true[i:i+24]
        yp = y_pred[i:i+24]
        a = K.abs(yt-yp)
        c = 113
        S = K.sum(yt)
        res += K.sum(a*yt/(S*c))
        cnt += 1
    return res/cnt

class Lookahead(keras.optimizers.Optimizer):
    def __init__(self, optimizer, sync_period=5, slow_step=0.5, **kwargs):
        super(Lookahead, self).__init__(**kwargs)
        self.optimizer = keras.optimizers.get(optimizer)
        with K.name_scope(self.__class__.__name__):
            self.sync_period = K.variable(sync_period, dtype='int64', name='sync_period')
            self.slow_step = K.variable(slow_step, name='slow_step')

    @property
    def lr(self):
        return self.optimizer.lr

    @lr.setter
    def lr(self, lr):
        self.optimizer.lr = lr

    @property
    def learning_rate(self):
        return self.optimizer.learning_rate

    @learning_rate.setter
    def learning_rate(self, learning_rate):
        self.optimizer.learning_rate = learning_rate

    @property
    def iterations(self):
        return self.optimizer.iterations

    def get_updates(self, loss, params):
        sync_cond = K.equal((self.iterations + 1) // self.sync_period * self.sync_period, (self.iterations + 1))
        slow_params = {p.name: K.variable(K.get_value(p), name='sp_{}'.format(i)) for i, p in enumerate(params)}
        update_names = ['update', 'update_add', 'update_sub']
        original_updates = [getattr(K, name) for name in update_names]
        setattr(K, 'update', lambda x, new_x: ('update', x, new_x))
        setattr(K, 'update_add', lambda x, new_x: ('update_add', x, new_x))
        setattr(K, 'update_sub', lambda x, new_x: ('update_sub', x, new_x))
        self.updates = self.optimizer.get_updates(loss, params)
        for name, original_update in zip(update_names, original_updates):
            setattr(K, name, original_update)
        slow_updates = []
        for i, update in enumerate(self.updates):
            if isinstance(update, tuple):
                name, x, new_x, adjusted = update + (update[-1],)
                update_func = getattr(K, name)
                if name == 'update_add':
                    adjusted = x + new_x
                if name == 'update_sub':
                    adjusted = x - new_x
                if x.name not in slow_params:
                    self.updates[i] = update_func(x, new_x)
                else:
                    slow_param = slow_params[x.name]
                    slow_param_t = slow_param + self.slow_step * (adjusted - slow_param)
                    slow_updates.append(K.update(slow_param, K.switch(
                        sync_cond,
                        slow_param_t,
                        slow_param,
                    )))
                    self.updates[i] = K.update(x, K.switch(
                        sync_cond,
                        slow_param_t,
                        adjusted,
                    ))
        slow_params = list(slow_params.values())
        self.updates += slow_updates
        self.weights = self.optimizer.weights + slow_params
        return self.updates

    def get_config(self):
        config = {
            'optimizer': keras.optimizers.serialize(self.optimizer),
            'sync_period': int(K.get_value(self.sync_period)),
            'slow_step': float(K.get_value(self.slow_step)),
        }
        base_config = super(Lookahead, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        optimizer = keras.optimizers.deserialize(config.pop('optimizer'))
        return cls(optimizer, **config)
    
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
target = pd.read_csv('./data/SolarPV_Elec_Problem.csv', header=None)
data = pd.read_csv('./data/features3.csv')#.iloc[0:-2,:] # 자료가 21시까지
# sub = pd.read_csv('./data/제출양식_복원값.xlsx')

In [ ]:
target['year'] = target[0].map(grap_year)
target['md'] = target[0].map(grap_date)
target['hour'] = target[0].map(grap_hour)
target = target.drop(0, axis=1)

In [ ]:
target

In [ ]:
target = target.fillna(-10)
y = target.groupby(['year', 'md', 'hour']).sum()[1].values[:8760]


In [ ]:
zero_col = ['rain', 'sunlight', 'sunpower', 'snow', 'windpower']
ukn_num = ['cloud2', 'cloud1']
ukn_cat = ['cloud3']
ctd = ['cloudheight', 'time']

categorical = [
    'winddirection', 'cloud1', 'cloud2', 'cloud3'
]

numerical = [
    'temperatur', 'rain', 'windpower', 'humidity', 'steampower', 'dew', 'pressure1', 'pressure2', 'sunpower', 'snow',
    'sijung', 'gourndtemperatur'
]

data[zero_col] = data[zero_col].fillna(0)
data[ukn_cat] = data[ukn_cat].fillna('unknown')
# data[categorical] = data[categorical].fillna('unknown')
data[ukn_num] = data[ukn_num].fillna(-1)
data[numerical] = data[numerical].fillna(0)
# data = data.drop(ctd, axis=1)


for c in categorical:
    lbe = LabelEncoder()
    data[c] = lbe.fit_transform(data[c])
    temp = pd.get_dummies(data[c], prefix=c)
    data = pd.concat([data, temp], axis=1).drop(c, axis=1)

    
d1 = data[data['code']==127] # 충주
d1.columns = [c+'_'+str(127) for c in d1.columns]
d1 = d1.drop('code_127', axis=1).reset_index().drop('index', axis=1)

d2 = data[data['code']==131] # 청주
d2.columns = [c+'_'+str(131) for c in d2.columns]
d2 = d2.drop('code_131', axis=1).reset_index().drop('index', axis=1)

d3 = data[data['code']==232] # 천안
d3.columns = [c+'_'+str(232) for c in d3.columns]
d3 = d3.drop('code_232', axis=1).reset_index().drop('index', axis=1)

data = pd.concat([d1, d2, d3], axis=1).iloc[:8760,:]

for c in data.columns:
    if data[c].std() == 0:
        data = data.drop(c, axis=1)
        
tot_num = []
for i in [127, 131, 232]:
    for c in numerical:
        tot_num.append(c+'_'+str(i))

for c in tot_num:
    if c in data.columns:
        data[c] /= data[c].max()
data

In [ ]:
steps = 24
tr_X = []
tr_y = []
val_X = []
val_y = []

y1 = y[:5136]
for i in range(steps, len(y1)):
    tr_X.append(y1[i-steps:i])
    tr_y.append(y1[i])
    
for i in range(24, 0, -1):
    val_X.append(tr_X[-i])
    val_y.append(tr_y[-i])
    
for _ in range(24): 
    tr_X.pop(-1)
    tr_y.pop(-1)
    
    
y1 = y[5160:6552]
for i in range(steps, len(y1)):
    tr_X.append(y1[i-steps:i])
    tr_y.append(y1[i])
    
for i in range(24, 0, -1):
    val_X.append(tr_X[-i])
    val_y.append(tr_y[-i])
    
for _ in range(24): 
    tr_X.pop(-1)
    tr_y.pop(-1)
    
    
y1 = y[6600:8040]
for i in range(steps, len(y1)):
    tr_X.append(y1[i-steps:i])
    tr_y.append(y1[i])

for i in range(24, 0, -1):
    val_X.append(tr_X[-i])
    val_y.append(tr_y[-i])
    
for _ in range(24): 
    tr_X.pop(-1)
    tr_y.pop(-1)
    
    
y1 = y[8088:8760]
for i in range(steps, len(y1)):
    tr_X.append(y1[i-steps:i])
    tr_y.append(y1[i])
    
for i in range(24, 0, -1):
    val_X.append(tr_X[-i])
    val_y.append(tr_y[-i])
    
for _ in range(24): 
    tr_X.pop(-1)
    tr_y.pop(-1)

In [ ]:
tr_X = np.array(tr_X)
tr_y = np.array(tr_y)
val_X = np.array(val_X)
val_y = np.array(val_y)

In [ ]:
def rolling(model, x):
    X = list(x)
    res = []
    for i in range(24):
        res.append(model.predict(np.expand_dims(X, 0)).flatten()[0])
        X.append(res[-1])
        X.pop(0)
    return res

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

In [ ]:
rf.fit(tr_X, tr_y)

In [ ]:
print(custom_loss(val_y[:24], rolling(rf, val_X[0])))
print(custom_loss(val_y[24:48], rolling(rf, val_X[24])))
print(custom_loss(val_y[48:72], rolling(rf, val_X[48])))
print(custom_loss(val_y[72:], rolling(rf, val_X[72])))

In [ ]:
custom_loss(val_y, pred)

In [ ]:
pred = rolling(rf, val_X[0]) + rolling(rf, val_X[24]) + rolling(rf, val_X[48]) 

In [ ]:
plt.plot(val_y[:-24], label='true')
plt.plot(pred, label='pred')
plt.title('RandomForest rolling')
plt.legend()
plt.show()

In [ ]:
pred = rf.predict(val_X)
plt.plot(val_y[:-24], label='true')
plt.plot(pred[:-24], label='pred')
plt.title('RandomForest direct')
plt.legend()
plt.show()

In [ ]:
def wave_block(filters,kernel_size,n):
    def f(x):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters=filters,
                    kernel_size=1, 
                    padding='same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters=filters,
                    kernel_size=kernel_size, 
                    padding='same',
                    activation = 'tanh',
                    dilation_rate=dilation_rate)(x)
            sigm_out = Conv1D(filters=filters,
                    kernel_size=kernel_size, 
                    padding='same',
                    activation = 'sigmoid',
                    dilation_rate=dilation_rate)(x)
            x = Multiply()([tanh_out,sigm_out])
            x = Conv1D(filters = filters,
                           kernel_size = 1,
                           padding='same')(x)
            # x = BatchNormalization()(x)
    #             x = SpatialDropout1D(0.2)(x)
            res_x = Add()([res_x,x])
        return res_x
    return f

def conv1d_block(x, fs, ks, activation=None):
    x = Conv1D(fs, ks, padding='causal', kernel_initializer='he_normal')(x)
    x = Activation(activation)(x)
    return x

In [ ]:
ttr_X = np.expand_dims(tr_X, -1)

In [ ]:
def build_flat():
    inputs = Input(shape = ttr_X.shape[1:])
    x1 = conv1d_block(inputs, 16, 2, mish)
    x1 = MaxPooling1D(2)(x1)
    x1 = conv1d_block(x1, 32, 2, mish)
    x1 = MaxPooling1D(2)(x1)
    x1 = Bidirectional(LSTM(32, return_sequences=True))(x1)
    x1 = Flatten()(x1)
    # x1 = Attention(24)(x1)

    x1_out = Dense(512, kernel_initializer='he_normal')(x1)
    x1_out = Activation(mish)(x1_out)
    x1_out = Dense(256, kernel_initializer='he_normal')(x1_out)
    x1_out = Activation(mish)(x1_out)
    x1_out = Dense(128, kernel_initializer='he_normal')(x1_out)
    x1_out = Activation(mish)(x1_out)
    x1_out = Dense(1, kernel_initializer='he_normal')(x1_out)

    x2 = conv1d_block(inputs, 16, 3, mish)
    x2 = MaxPooling1D(2)(x2)
    x2 = conv1d_block(x2, 32, 3, mish)
    x2 = MaxPooling1D(2)(x2)
    x2 = Bidirectional(LSTM(32, return_sequences=True))(x2)
    x2 = Flatten()(x2)
    # x2 = Attention(24)(x2)

    x2_out = Dense(512, kernel_initializer='he_normal')(x2)
    x2_out = Activation(mish)(x2_out)
    x2_out = Dense(256, kernel_initializer='he_normal')(x2_out)
    x2_out = Activation(mish)(x2_out)
    x2_out = Dense(128, kernel_initializer='he_normal')(x2_out)
    x2_out = Activation(mish)(x2_out)
    x2_out = Dense(1)(x2_out)

    x3 = conv1d_block(inputs, 16, 5, mish)
    x3 = MaxPooling1D(2)(x3)
    x3 = conv1d_block(x3, 32, 5, mish)
    x3 = MaxPooling1D(2)(x3)
    x3 = Bidirectional(LSTM(32, return_sequences=True))(x3)
    x3 = Flatten()(x3)
    # x3 = Attention(24)(x3)

    x3_out = Dense(512, kernel_initializer='he_normal')(x3)
    x3_out = Activation(mish)(x3_out)
    x3_out = Dense(256, kernel_initializer='he_normal')(x3_out)
    x3_out = Activation(mish)(x3_out)
    x3_out = Dense(128, kernel_initializer='he_normal')(x3_out)
    x3_out = Activation(mish)(x3_out)
    x3_out = Dense(1)(x3_out)

    x4 = conv1d_block(inputs, 16, 7, mish)
    x4 = MaxPooling1D(2)(x4)
    x4 = conv1d_block(x4, 32, 7, mish)
    x4 = MaxPooling1D(2)(x4)
    x4 = Bidirectional(LSTM(32, return_sequences=True))(x4)
    x4 = Flatten()(x4)
    # x4 = Attention(24)(x4)

    x4_out = Dense(512, kernel_initializer='he_normal')(x4)
    x4_out = Activation(mish)(x4_out)
    x4_out = Dense(256, kernel_initializer='he_normal')(x4_out)
    x4_out = Activation(mish)(x4_out)
    x4_out = Dense(128, kernel_initializer='he_normal')(x4_out)
    x4_out = Activation(mish)(x4_out)
    x4_out = Dense(1)(x4_out)

    x = Add()([x1, x2, x3, x4])
    x = Dense(512, kernel_initializer='he_normal')(x)
    x = Activation(mish)(x)
    x = Dense(256, kernel_initializer='he_normal')(x)
    x = Activation(mish)(x)
    x = Dense(128, kernel_initializer='he_normal')(x)
    x = Activation(mish)(x)
    x = Dense(1)(x)

    nn = Model(inputs, [x, x1_out,x2_out,x3_out, x4_out])
    return nn


def build_att():
    inputs = Input(shape = ttr_X.shape[1:])
    x1 = conv1d_block(inputs, 16, 2, mish)
    x1 = MaxPooling1D(2)(x1)
    x1 = conv1d_block(x1, 32, 2, mish)
    x1 = MaxPooling1D(2)(x1)
    x1 = Bidirectional(LSTM(32, return_sequences=True))(x1)
#     x1 = Flatten()(x1)
    x1 = Attention(12)(x1)

    x1_out = Dense(64, kernel_initializer='he_normal')(x1)
    x1_out = Activation(mish)(x1_out)
    x1_out = Dense(32, kernel_initializer='he_normal')(x1_out)
    x1_out = Activation(mish)(x1_out)
    x1_out = Dense(1, kernel_initializer='he_normal')(x1_out)

    x2 = conv1d_block(inputs, 16, 3, mish)
    x2 = MaxPooling1D(2)(x2)
    x2 = conv1d_block(x2, 32, 3, mish)
    x2 = MaxPooling1D(2)(x2)
    x2 = Bidirectional(LSTM(32, return_sequences=True))(x2)
#     x2 = Flatten()(x2)
    x2 = Attention(12)(x2)

    x2_out = Dense(64, kernel_initializer='he_normal')(x2)
    x2_out = Activation(mish)(x2_out)
    x2_out = Dense(32, kernel_initializer='he_normal')(x2_out)
    x2_out = Activation(mish)(x2_out)
#     x2_out = Dense(128, kernel_initializer='he_normal')(x2_out)
#     x2_out = Activation(mish)(x2_out)
    x2_out = Dense(1)(x2_out)

    x3 = conv1d_block(inputs, 16, 5, mish)
    x3 = MaxPooling1D(2)(x3)
    x3 = conv1d_block(x3, 32, 5, mish)
    x3 = MaxPooling1D(2)(x3)
    x3 = Bidirectional(LSTM(32, return_sequences=True))(x3)
#     x3 = Flatten()(x3)
    x3 = Attention(12)(x3)

    x3_out = Dense(64, kernel_initializer='he_normal')(x3)
    x3_out = Activation(mish)(x3_out)
    x3_out = Dense(32, kernel_initializer='he_normal')(x3_out)
    x3_out = Activation(mish)(x3_out)
    x3_out = Dense(1)(x3_out)

    x4 = conv1d_block(inputs, 16, 7, mish)
    x4 = MaxPooling1D(2)(x4)
    x4 = conv1d_block(x4, 32, 7, mish)
    x4 = MaxPooling1D(2)(x4)
    x4 = Bidirectional(LSTM(32, return_sequences=True))(x4)
#     x4 = Flatten()(x4)
    x4 = Attention(12)(x4)

    x4_out = Dense(64, kernel_initializer='he_normal')(x4)
    x4_out = Activation(mish)(x4_out)
    x4_out = Dense(32, kernel_initializer='he_normal')(x4_out)
    x4_out = Activation(mish)(x4_out)
#     x4_out = Dense(128, kernel_initializer='he_normal')(x4_out)
#     x4_out = Activation(mish)(x4_out)
    x4_out = Dense(1)(x4_out)

    x = Add()([x1, x2, x3, x4])
    x = Dense(64, kernel_initializer='he_normal')(x)
    x = Activation(mish)(x)
    x = Dense(32, kernel_initializer='he_normal')(x)
    x = Activation(mish)(x)
    x = Dense(1)(x)

    nn = Model(inputs, [x, x1_out,x2_out,x3_out, x4_out])
    return nn


def build_wav():
    inputs = Input(shape = ttr_X.shape[1:])
    x1 = wave_block(16, 3, 5)(inputs)
    x1 = MaxPooling1D(2)(x1)
    x1 = wave_block(32, 3, 3)(x1)
    x1 = MaxPooling1D(2)(x1)
    x1 = Bidirectional(LSTM(32, return_sequences=True))(x1)
    x1 = Attention(12)(x1)

    x = Dense(64, kernel_initializer='he_normal')(x1)
    x = Activation(mish)(x)
    x = Dense(32, kernel_initializer='he_normal')(x)
    x = Activation(mish)(x)
    x = Dense(1)(x)

    nn = Model(inputs, x)
    return nn


In [ ]:
nn = build_wav()
nn.summary()

In [ ]:
nn.compile(loss='mse', optimizer=Lookahead(optimizers.Adam()))
nn.fit(ttr_X/113, tr_y/113,# [tr_y/113]*5
      epochs=200,
       validation_split=0.2,
       batch_size=32,
      callbacks=[es, lrs])

In [ ]:
custom_loss(val_y, (nn.predict(np.expand_dims(val_X/113, -1))).flatten()*113)

In [ ]:
def rolling_nn(model, x):
    X = list(x)
    res = []
    for i in range(24):
        res.append(model.predict(np.array(np.clip(X, 0, 1)).reshape((1,48,1)))[0].flatten()[0]*113)
        X.append(res[-1])
        X.pop(0)
    return res

In [ ]:
print(custom_loss(val_y[:24], rolling_nn(nn, np.expand_dims(val_X[0]/113, -1))))
print(custom_loss(val_y[24:48], rolling_nn(nn, np.expand_dims(val_X[24]/113, -1))))
print(custom_loss(val_y[48:72], rolling_nn(nn, np.expand_dims(val_X[48]/113, -1))))